In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch, helpers
import numpy as np
import pandas as pd
import pprint
from __future__ import division
from __future__ import print_function
from datetime import datetime
import calendar
from zipfile import ZipFile
import os
import sys
import dateutil
import zipfile
import math
import time
import itertools
es = Elasticsearch(['atlas-kibana.mwt2.org:9200'])

In [2]:
def calcMathisModel_outMBitsPerSec(packetloss, MSS, RTT, C=math.sqrt(3/2)):
    '''
    Returns the result of mathis model in MBits/s.
    It does work with np.arrays.
    '''
    outVal = (MSS/RTT)*C/np.sqrt(packetloss)
    return outVal

In [3]:
def getRealTimeMathisEstimation(src_data, dest_data):
    
    data = pd.DataFrame()
    
    response_size = 1000
    es_sorting = "timestamp:desc"
    es_query_forth = \
    {
        "query": {
            "filtered": {
                "query": {
                    "match_all": {}
                },
                "filter": {
                    "bool": {
                        "must":[
                            { 'term': { 'src': src_data["latency"] } },
                            { 'term': { 'dest': dest_data["latency"] } },
                        ]
                    }
                }
            }
        }
    }
    es_query_back = \
    {
        "query": {
            "filtered": {
                "query": {
                    "match_all": {}
                },
                "filter": {
                    "bool": {
                        "must":[
                            { 'term': { 'src': dest_data["latency"] } },
                            { 'term': { 'dest': src_data["latency"] } },
                        ]
                    }
                }
            }
        }
    }

    es_index = "network_weather_2-*"

    # Collect all needed data raw data
    event_type = 'latency'
    response_latency_forth = es.search(index=es_index, body=es_query_forth, size=response_size, sort=es_sorting, doc_type = event_type, request_timeout=30)
    response_latency_back = es.search(index=es_index, body=es_query_back, size=response_size, sort=es_sorting, doc_type = event_type, request_timeout=30)
    event_type = 'packet_loss_rate'
    response_packetloss = es.search(index=es_index, body=es_query_forth, size=response_size, sort=es_sorting, doc_type = event_type, request_timeout=30)

    # extract data
    for dataPoint in response_latency_forth['hits']['hits']:
        time = dateutil.parser.parse(dataPoint['_source']['timestamp'])
        timestamp_epoch = calendar.timegm(time.timetuple())
        data.set_value(timestamp_epoch, 'delay_mean_forth', dataPoint['_source']['delay_mean'])

    for dataPoint in response_latency_back['hits']['hits']:
        time = dateutil.parser.parse(dataPoint['_source']['timestamp'])
        timestamp_epoch = calendar.timegm(time.timetuple())
        data.set_value(timestamp_epoch, 'delay_mean_back', dataPoint['_source']['delay_mean'])

    for dataPoint in response_packetloss['hits']['hits']:
        time = dateutil.parser.parse(dataPoint['_source']['timestamp'])
        timestamp_epoch = calendar.timegm(time.timetuple())
        data.set_value(timestamp_epoch, 'packet_loss', dataPoint['_source']['packet_loss'])

    # analyze data
    currentRow = 0
    # this can be sped up by using data.itertuples() instead of data.iterrows()
    for row in data.iterrows():
        if np.isfinite(row[1]).all() == True:
            currentRow = row
            break;
    if currentRow == 0:
        raise StandardError("Could not retrieve enough data to evaluate.")

    # asuming jumbo frames:
    MSS = 8860
    mathisEstimation = 0
    if currentRow[1][2] == 0.0:
        return {'estimation Mbit/s': -1, 'timestamp': currentRow[0]}
    else:
        mathisEstimation = calcMathisModel_outMBitsPerSec(currentRow[1][2], MSS, currentRow[1][0]+currentRow[1][1])
        return {'estimation Mbit/s': mathisEstimation, 'timestamp': currentRow[0]}
    


In [4]:
CERN_data = {'name': "CERN", "latency": "128.142.223.247", "throughput": "128.142.223.246"}
RAL_data = {'name': "RAL", "latency": "130.246.176.109", "throughput": "130.246.176.110"}
PIC_data = {'name': "PIC", "latency": "193.109.172.188", "throughput": "193.109.172.187"}
TRIUMF_data = {'name': "TRIUMF", "latency": "206.12.9.2", "throughput": "206.12.9.1"}
BNL_data = {'name': "BNL", "latency": "192.12.15.26", "throughput": "192.12.15.23"}
KIT_data = {'name': "KIT", "latency": "192.108.47.12", "throughput": "192.108.47.6"}

site_list = [CERN_data, RAL_data, PIC_data, TRIUMF_data, BNL_data, KIT_data]

src_data = CERN_data
dest_data = TRIUMF_data

#while (True):
#    getRealTimeMathisEstimation(src_data, dest_data)
#    time.sleep(30)


In [5]:
while 1:
    mathisData = {}
    for sites in itertools.combinations(site_list, 2):
        sites_local = sites
        for i in range(2):
            print(".", end='')
            if sites_local[0]['name'] not in mathisData:
                mathisData[sites_local[0]['name']] = {}
            mathisData[sites_local[0]['name']][sites_local[1]['name']] = getRealTimeMathisEstimation(sites_local[0], sites_local[1])
            sites_local = (sites[1], sites[0])
    print(" ")
    mathisData

.

AttributeError: 'int' object has no attribute 'read'